In [1]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np


In [2]:
#IMPORT THE INPUT FILE

df_input = pd.read_csv('/Users/emirkabasoglu/Downloads/hubspot-crm-exports-lead-report-eloomi-2023-2024-09-02.csv')

In [3]:
#Check if it's the correct dataframe


df_input.head(3)

,Record ID,First Name,Last Name,Market,Lifecycle Stage,Lead source (Only for Inbound),Number of Employees,Lead status vol2,Country,Deal pipeline,Email,Contact owner,Inbound Lead date (Do not change),Create Date,Lead type,Company Name,Associated Deal,Associated Deal IDs
0,53778603617,Phil,Thorne,UK,Marketing Qualified Lead,PPC - Google AdWords,1-49,Meeting booked,United Kingdom,NaN,phil.thorne@quorso.com,Viktoria Vas,2024-08-31,2024-08-31 23:55,Inbound,Quorso,NaN,NaN
1,53774118448,M,Grent,NORBEN,Lead,PPC - Facebook,1-49,New,Netherlands,NaN,mail@marliesgrent.nl,NaN,2024-08-31,2024-08-31 23:13,Inbound,NaN,NaN,NaN
2,53774298914,Goodelife,HR,US,Lead,PPC - Instagram,1-49,New,United States,NaN,info@goodelife.net,NaN,2024-08-31,2024-08-31 23:04,Inbound,NaN,NaN,NaN


In [4]:
#Assing number of employees

def assing_no_employees(df):
    if df['Number of Employees'] == '1-49':
        return '0-49'
    elif df['Number of Employees'] == '50-99':
        return '50-99'
    elif df['Number of Employees'] == '100-249':
        return '100-249'
    elif df['Number of Employees'] == '250-499':
        return '250-499'
    elif df['Number of Employees'] == '500-999':
        return '500-999'
    elif df['Number of Employees'] == '1000-4999':
        return '1000-4999'
    elif df['Number of Employees'] == '5000+':
        return '5000+'
    else:
        return 'Unknown'

df_input['Number_of_employees'] = df_input.apply(assing_no_employees, axis=1)

In [5]:
#Assing a region

norben = ['Denmark', 'Norway', 'Sweden', 'Finland', 'Iceland', 'Netherlands', 'Belgium', 'Luxembourg']
dach = ['Switzerland','Germany', 'Austria']
us = ['United States', 'Canada']
uk = ['United Kingdom', 'Ireland']

def assing_region(df):
    if df['Country'] in norben:
        return 'NorBen'
    elif df['Country'] in dach:
        return 'DACH'
    elif df['Country'] in us:
        return 'US'
    elif df['Country'] in uk:
        return 'UK'
    else :
        return 'ROW'

df_input['Region'] = df_input.apply(assing_region, axis=1)

In [6]:
#Assing MQL Lead 

mql = ['Marketing Qualified Lead','Sales Qualified Lead','Opportunity','Customer']

def assing_mql(df):
    if df['Lifecycle Stage'] in mql:
        return 1
    else:
        return 0

    
df_input['mql_status'] = df_input.apply(assing_mql,axis=1)

In [7]:
#Checking how many mql leads are in the dataframe

df_input['mql_status'].value_counts()

mql_status
0    1070
1     339
Name: count, dtype: int64

In [8]:
#Assigning MQL Leads with a Demo Booked

mql_demo_booked = ['Sales Qualified Lead','Opportunity','Customer']

def assing_mql_booked_demo(df):
    if df['Lifecycle Stage'] in mql_demo_booked or ( df['Lifecycle Stage']=='Marketing Qualified Lead' and pd.notnull(df['Associated Deal'])):
        return 1
    else:
        return 0
    
    
df_input['mql_demo_booked'] = df_input.apply(assing_mql_booked_demo,axis=1)
    
    

In [9]:
#Checking how many mql leads with a demo booked are in the dataframe

df_input['mql_demo_booked'].value_counts()

mql_demo_booked
0    1305
1     104
Name: count, dtype: int64

In [10]:
#Assigning SQL Leads

sql = ['Sales Qualified Lead','Opportunity','Customer']

def assing_sql(df):
    if df['Lifecycle Stage'] in sql:
        return 1
    else:
        return 0
    
    
df_input['sql_status'] = df_input.apply(assing_sql,axis=1)
    
    

In [11]:
#Checking how many sql leads are in the dataframe

df_input['sql_status'].value_counts()

sql_status
0    1344
1      65
Name: count, dtype: int64

In [12]:
#Assigning Oppurtunity Leads

oppurtunity = ['Opportunity','Customer']

def assing_oppurtunity(df):
    if df['Lifecycle Stage'] in oppurtunity:
        return 1
    else:
        return 0
    
    
df_input['Opportunity_status'] = df_input.apply(assing_oppurtunity,axis=1)
    
    

In [13]:
#Checking how many oppurtunties are in the dataframe

df_input['Opportunity_status'].value_counts()

Opportunity_status
0    1399
1      10
Name: count, dtype: int64

In [14]:
#Assigning Won Leads 

won = ['Customer']

def assing_won_deals(df):
    if df['Lifecycle Stage'] in won:
        return 1
    else:
        return 0
    
    
df_input['won_status'] = df_input.apply(assing_won_deals,axis=1)
    
    

In [15]:
#Checking how many oppurtunties are in the dataframe

df_input['won_status'].value_counts()

won_status
0    1408
1       1
Name: count, dtype: int64

In [16]:
#Counting the leads,demos,sql's,oppurtunities,customers and grouping them


def calculation_grouping(df):
    names = {
        'Total Leads': df['Record ID'].count(),                                           #Total Leads
        '# of MQLs': df['mql_status'].sum(),                                              # of MQLs
        'CVR lead to MQL': df['mql_status'].sum()/df['Record ID'].count(),                #CVR lead to MQL 
        'CVR lead to MQLs with a demo booked': df['mql_demo_booked'].sum()/df['Record ID'].count(), #CVR lead to MQL with demo booked
        '# of MQLs with a booked demo': df['mql_demo_booked'].sum(),                      # of MQL with demo booked
        'CVR MQL to SQL': df['sql_status'].sum()/df['mql_status'].sum(),                  #CVR lead to SQL
        '# of SQLs':df['sql_status'].sum(),                                               ## SQL
        'CVR SQL to Opportunities':df['Opportunity_status'].sum()/df['sql_status'].sum(), #CVR lead to Opportunity
        '# of Opportunities':df['Opportunity_status'].sum(),                              ## OPP
        'CVR Opportunities to Won': df['won_status'].sum()/df['Opportunity_status'].sum(),#CVR OPP to WON
        '# of Won': df['won_status'].sum(),                                               #Customer
        'CVR of SQL to Won':  df['won_status'].sum()/ df['sql_status'].sum()              #"Closing rate SQL to WON
            }
    return pd.Series(names)

df_leads_grp = df_input.groupby(by = ['Lead source (Only for Inbound)','Region'#,'Number_of_employees'
                                      ]).apply(calculation_grouping)



<ipython-input-16-1d200580473c>:13: RuntimeWarning: invalid value encountered in scalar divide
  'CVR SQL to Opportunities':df['Opportunity_status'].sum()/df['sql_status'].sum(), #CVR lead to Opportunity
<ipython-input-16-1d200580473c>:15: RuntimeWarning: invalid value encountered in scalar divide
  'CVR Opportunities to Won': df['won_status'].sum()/df['Opportunity_status'].sum(),#CVR OPP to WON
<ipython-input-16-1d200580473c>:17: RuntimeWarning: invalid value encountered in scalar divide
  'CVR of SQL to Won':  df['won_status'].sum()/ df['sql_status'].sum()              #"Closing rate SQL to WON
<ipython-input-16-1d200580473c>:11: RuntimeWarning: invalid value encountered in scalar divide
  'CVR MQL to SQL': df['sql_status'].sum()/df['mql_status'].sum(),                  #CVR lead to SQL
<ipython-input-16-1d200580473c>:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns wil

In [17]:
#Rounding up the values to 2 decimals & Filling Nan with 0's

df_leads_grp = np.round(df_leads_grp, decimals=2)
df_leads_grp = df_leads_grp.fillna(0)
df_leads_grp

Total Leads  # of MQLs  \
Lead source (Only for Inbound)     Region                           
Dayforce customer                  US              1.0        1.0   
Direct - Website                   DACH            1.0        1.0   
                                   NorBen          5.0        3.0   
                                   ROW            11.0        7.0   
                                   UK             30.0       18.0   
                                   US             66.0       47.0   
Organic - Bing                     UK              1.0        1.0   
                                   US              2.0        2.0   
Organic - Facebook                 NorBen          1.0        0.0   
                                   UK              4.0        0.0   
                                   US              2.0        0.0   
Organic - Google                   DACH            1.0        1.0   
                                   NorBen          5.0        3.0   
                                   ROW             8.0        6.0   
                                   UK             11.0        7.0   
                                   US             13.0       10.0   
Organic - Other                    ROW             1.0        0.0   
                                   US              1.0        1.0   
Other                              NorBen          1.0        0.0   
                                   UK              1.0        0.0   
PPC - Bing                         NorBen          1.0        1.0   
                                   UK              4.0        3.0   
                                   US             20.0       20.0   
PPC - Facebook                     NorBen         45.0        5.0   
                                   ROW             1.0        0.0   
                                   UK             55.0        6.0   
                                   US             85.0       12.0   
PPC - Google AdWords               NorBen          7.0        7.0   
                                   ROW             1.0        1.0   
                                   UK             41.0       34.0   
                                   US             42.0       38.0   
PPC - Instagram                    DACH            2.0        0.0   
                                   NorBen        107.0       13.0   
                                   ROW             5.0        0.0   
                                   UK             91.0       11.0   
                                   US            203.0       19.0   
PPC - LinkedIN                     NorBen          1.0        0.0   
                                   ROW             1.0        1.0   
                                   UK              1.0        0.0   
Referral - Other                   DACH            1.0        1.0   
                                   ROW             2.0        0.0   
                                   US             10.0        1.0   
Review DB - Capterra               UK              1.0        1.0   
Review DB - People Managing People ROW             1.0        1.0   
                                   US              1.0        1.0   
Review DB - Software Advice PPL    US             55.0       55.0   
Review DB - Sourceforge            DACH            1.0        0.0   
                                   US              1.0        0.0   
Sponsored content                  DACH            4.0        0.0   
                                   ROW            23.0        0.0   
                                   US            430.0        0.0   

                                           CVR lead to MQL  \
Lead source (Only for Inbound)     Region                    
Dayforce customer                  US                 1.00   
Direct - Website                   DACH               1.00   
                                   NorBen             0.60   
                                   ROW                0.64   
             

In [18]:
#Finding the region and number of employees for all leads by changing the inbound source to Total for all of them

df_total = df_input
df_total['Lead source (Only for Inbound)'] = 'Total'

df_leads_total = df_total.groupby(by = ['Lead source (Only for Inbound)','Region'#,'Number_of_employees'
                                        ]).apply(calculation_grouping)

df_leads_total = np.round(df_leads_total, decimals=2)

df_leads_total = df_leads_total.fillna(0)

df_leads_total

<ipython-input-16-1d200580473c>:15: RuntimeWarning: invalid value encountered in scalar divide
  'CVR Opportunities to Won': df['won_status'].sum()/df['Opportunity_status'].sum(),#CVR OPP to WON
<ipython-input-18-d971796627d2>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_leads_total = df_total.groupby(by = ['Lead source (Only for Inbound)','Region'#,'Number_of_employees'


Total Leads  # of MQLs  \
Lead source (Only for Inbound) Region                           
Total                          DACH           10.0        3.0   
                               NorBen        173.0       32.0   
                               ROW            54.0       16.0   
                               UK            240.0       81.0   
                               US            932.0      207.0   

                                       CVR lead to MQL  \
Lead source (Only for Inbound) Region                    
Total                          DACH               0.30   
                               NorBen             0.18   
                               ROW                0.30   
                               UK                 0.34   
                               US                 0.22   

                                       CVR lead to MQLs with a demo booked  \
Lead source (Only for Inbound) Region                                        
Total                          DACH                                   0.10   
                               NorBen                                 0.03   
                               ROW                                    0.07   
                               UK                                     0.10   
                               US                                     0.08   

                                       # of MQLs with a booked demo  \
Lead source (Only for Inbound) Region                                 
Total                          DACH                             1.0   
                               NorBen                           5.0   
                               ROW                              4.0   
                               UK                              24.0   
                               US                              70.0   

                                       CVR MQL to SQL  # of SQLs  \
Lead source (Only for Inbound) Region                              
Total                          DACH              0.33        1.0   
                               NorBen            0.06        2.0   
                               ROW               0.12        2.0   
                               UK                0.21       17.0   
                               US                0.21       43.0   

                                       CVR SQL to Opportunities  \
Lead source (Only for Inbound) Region                             
Total                          DACH                        0.00   
                               NorBen                      0.00   
                               ROW                         0.00   
                               UK                          0.18   
                               US                          0.16   

                                       # of Opportunities  \
Lead source (Only for Inbound) Region                       
Total                          DACH                   0.0   
                               NorBen                 0.0   
                               ROW                    0.0   
                               UK                     3.0   
                               US                     7.0   

                                       CVR Opportunities to Won  # of Won  \
Lead source (Only for Inbound) Region                                       
Total                          DACH                        0.00       0.0   
                               NorBen                      0.00       0.0   
                               ROW                         0.00       0.0   
                               UK                          0.00       0.0   
                               US                          0.14       1.0   

                                       CVR of SQL to Won  
Lead source (Only for Inbound) Region                     
Total                          DACH                 0.00  
                               NorBen  

In [19]:
#Concating to 2 dataframe after they have been grouped by

df_final = pd.concat([df_leads_total,df_leads_grp])
df_final

Total Leads  # of MQLs  \
Lead source (Only for Inbound)     Region                           
Total                              DACH           10.0        3.0   
                                   NorBen        173.0       32.0   
                                   ROW            54.0       16.0   
                                   UK            240.0       81.0   
                                   US            932.0      207.0   
Dayforce customer                  US              1.0        1.0   
Direct - Website                   DACH            1.0        1.0   
                                   NorBen          5.0        3.0   
                                   ROW            11.0        7.0   
                                   UK             30.0       18.0   
                                   US             66.0       47.0   
Organic - Bing                     UK              1.0        1.0   
                                   US              2.0        2.0   
Organic - Facebook                 NorBen          1.0        0.0   
                                   UK              4.0        0.0   
                                   US              2.0        0.0   
Organic - Google                   DACH            1.0        1.0   
                                   NorBen          5.0        3.0   
                                   ROW             8.0        6.0   
                                   UK             11.0        7.0   
                                   US             13.0       10.0   
Organic - Other                    ROW             1.0        0.0   
                                   US              1.0        1.0   
Other                              NorBen          1.0        0.0   
                                   UK              1.0        0.0   
PPC - Bing                         NorBen          1.0        1.0   
                                   UK              4.0        3.0   
                                   US             20.0       20.0   
PPC - Facebook                     NorBen         45.0        5.0   
                                   ROW             1.0        0.0   
                                   UK             55.0        6.0   
                                   US             85.0       12.0   
PPC - Google AdWords               NorBen          7.0        7.0   
                                   ROW             1.0        1.0   
                                   UK             41.0       34.0   
                                   US             42.0       38.0   
PPC - Instagram                    DACH            2.0        0.0   
                                   NorBen        107.0       13.0   
                                   ROW             5.0        0.0   
                                   UK             91.0       11.0   
                                   US            203.0       19.0   
PPC - LinkedIN                     NorBen          1.0        0.0   
                                   ROW             1.0        1.0   
                                   UK              1.0        0.0   
Referral - Other                   DACH            1.0        1.0   
                                   ROW             2.0        0.0   
                                   US             10.0        1.0   
Review DB - Capterra               UK              1.0        1.0   
Review DB - People Managing People ROW             1.0        1.0   
                                   US              1.0        1.0   
Review DB - Software Advice PPL    US             55.0       55.0   
Review DB - Sourceforge            DACH            1.0        0.0   
                                   US              1.0        0.0   
Sponsored content                  DACH            4.0        0.0   
                                   ROW            23.0        0.0   
                                   US            430.0        0.0   

                                        

In [20]:
df_final.to_excel('Lead Report.xlsx')